In [3]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [6]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 5.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [7]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 129 kB/s 


In [8]:
!pip install pyradiomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 4.3 MB/s 
     |████████████████████████████████| 109 kB 53.4 MB/s 
     |████████████████████████████████| 500 kB 34.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a442a8c8af4f9d5b8e263bb28e7fd6a65e20cc7a2b1b3aa5a57453747f320035
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [8]:
%%capture
!pip install dicomweb-client

In [9]:
import os
import glob
import numpy as np
import shutil
from google.cloud import storage
import nrrd
import SimpleITK as sitk
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap

In [10]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

import csv
import pandas as pd

In [14]:
%%bigquery --project=bwh-midrc-rapid-res-1655321320 ct_limited_open_a1_r1

WITH
  nlst_instances_per_series AS (
    SELECT
      DISTINCT(StudyInstanceUID),
      SeriesInstanceUID,
      COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
      MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
      MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
      MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
      MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
      STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer
    FROM
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
    WHERE
      (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
    GROUP BY
      StudyInstanceUID,
      SeriesInstanceUID
      ), 
  nlst_values_per_series AS (
    SELECT 
    ANY_VALUE(dicom_all.PatientID) AS PatientID,
    dicom_all.SeriesInstanceUID,
    ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
    ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
    ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_instances_per_series
  ON
    dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
  WHERE
    min_SliceThickness >= 1.5 
    AND max_SliceThickness <= 3.5 
    AND nlst_instances_per_series.num_instances > 100
    AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
    AND nlst_instances_per_series.orientation_count = 1
    AND has_localizer = "false"
  GROUP BY
    SeriesInstanceUID
  )
  SELECT 
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    dicom_all.SeriesInstanceUID,
    dicom_all.SOPInstanceUID,
    dicom_all.collection_id,
    dicom_all.PatientAge,
    dicom_all.PatientWeight,
    nlst_values_per_series.num_instances,
    nlst_values_per_series.SliceThickness,
    nlst_values_per_series.PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_values_per_series 
  ON
    dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID

In [9]:
def download_series(study_instance_uid, series_instance_uid, sop_instance_uids, dest_dir):
  import pydicom
  token = !gcloud auth print-access-token
  token = token[0]

  PROJECT_ID="bwh-midrc-rapid-res-1655321320"
  REGION="us-central1"

  DATASET_ID="midrc"
  DICOM_STORE_ID="midrc-dicom"
  
  my_project = "bwh-midrc-rapid-res-1655321320"
  location = "us-central1"
  dataset_id = "midrc"
  dicom_store_id = "midrc-dicom"

  url = f"https://healthcare.googleapis.com/v1/projects/{my_project}/locations/{location}/datasets/{dataset_id}/dicomStores/{dicom_store_id}/dicomWeb"
  headers = {
      "Authorization" : "Bearer %s" % token
  }

  import dicomweb_client

  client = dicomweb_client.api.DICOMwebClient(url, headers=headers)

  idx=0
  for sop_instance_uid in sop_instance_uids:
    retrievedInstance = client.retrieve_instance(
                study_instance_uid=study_instance_uid,
                series_instance_uid=series_instance_uid,
                sop_instance_uid=sop_instance_uid)
    pydicom.filewriter.write_file(f"{dest_dir}/file{idx}.dcm", retrievedInstance)
    idx+=1

In [10]:
def file_exists_in_bucket(project_name, bucket_name, file_gs_uri):
  
  """
  Check whether a file exists in the specified Google Cloud Storage Bucket.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - file GS URI
  
  Returns:
    file_exists : boolean variable, True if the file exists in the specified,
                  bucket, at the specified location; False if it doesn't.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  bucket_gs_url = "gs://%s/"%(bucket_name)
  path_to_file_relative = file_gs_uri.split(bucket_gs_url)[-1]

  print("Searching %s for: \n%s\n"%(bucket_gs_url, path_to_file_relative))

  file_exists = bucket.blob(path_to_file_relative).exists(storage_client)
  
  return file_exists

In [14]:
import logging
urllib3_logger = logging.getLogger('urllib3')
urllib3_logger.setLevel(logging.CRITICAL) # suppress messages upon download

In [1]:
from google.colab import files
uploaded = files.upload()

Saving param_ct.yaml to param_ct.yaml


In [11]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path) #, delimiter='/') #don't search in non-axial sub-folder
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    bs = bn.split('.')
    if bs[-1] == 'json':
      #print(bn)
      bss = bn.split('/')[1]
      #print(bss)
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [23]:
print(len(series_instance_uids))

1196


In [11]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    (r, ext) = os.path.splitext(bn)
    #print(r,ext)
    if ext == '.json':
      u = os.path.split(r)[-1]
      #print(u)
      series_instance_uids.append(u)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [12]:
print(len(series_instance_uids))

495


In [13]:
df = ct_limited_open_a1_r1
k=0

good_seriesuids = []

#series_instance_uids = ['1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715']

for seriesuid in series_instance_uids:
  #print(seriesuid)
  studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
  if len(studyuids)>0:
    k=k+1
    #print(seriesuid)
    studyuid = studyuids[0]
    sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()
    print(len(sops))
    #download_series(studyuid, seriesuid, sops, path_downloaded)
    good_seriesuids.append(seriesuid)
print(k)

NameError: ignored

In [15]:
bucket_name, bucket_path

('midrc-analysis-bwh', 'bpr-results/')

In [24]:
# process files through pyradiomics
# step 1, make a list of seriesuid's in the bucket
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name)
s = []
k=0
for b in blobs:
  #print('raw', b.name)
  p = os.path.split(b.name)
  #print(p)
  # skip non-axial and folders
  # temporarily autoremove any slashes from bucket_path
  if p[0]==bucket_path.replace('/','') and p[1].split('.')[-1]=='nrrd':
    k +=1
    seriesuid_tmp = '.'.join(p[1].split('.')[:-1])
    s.append(seriesuid_tmp)
    if k%10==0:
      print(k,' ', end='')
    if k%120==0:
        print('')
seriesuid_list = list(set([x.split('_')[1] for x in s ]))
print('\n',len(seriesuid_list))

10  20  30  40  50  60  70  80  90  100  110  120  
130  140  150  160  170  180  190  200  210  220  230  240  
250  260  270  280  290  300  310  320  330  340  350  360  
370  380  390  400  410  420  430  440  450  460  470  480  
490  500  510  520  530  540  550  560  570  580  590  600  
610  620  630  640  650  660  670  680  690  700  710  720  
730  740  750  760  770  780  790  800  810  820  830  840  
850  860  870  880  890  900  910  920  930  940  950  960  
970  980  
 494


In [27]:
# Processing series

path_downloaded = '/content/downloaded_data'
path_nifti = '/content/nifti_data'
path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'

good_seriesuids = [seriesuid_list[0]]

k=0
for seriesuid in good_seriesuids:
  k+=1
  print(k)

  # clean up from previous iterations and recreate temp directories
  if 1:
    for x in [path_downloaded, path_nifti, path_json, path_nrrd]:
      if os.path.isdir(x):
        try:
          shutil.rmtree(x)
        except OSError as err:
          print("Error: %s : %s" % (x, err.strerror))  
      os.mkdir(x)

  file_gs_uri_seg = "gs://%s/%sseg_%s.nrrd" % (bucket_name, bucket_path, seriesuid) # "/"  exists in the path
  file_gs_uri_ct = "gs://%s/%sct_%s.nrrd" % (bucket_name, bucket_path, seriesuid)
  
  cmd1 = "gsutil cp %s %s" % (file_gs_uri_seg, path_nrrd)
  print(cmd1)
  os.system(cmd1)
  
  cmd2 = "gsutil cp %s %s" % (file_gs_uri_ct, path_nrrd)
  print(cmd2)
  os.system(cmd2)
  
  fn_seg_nrrd = os.path.join(path_nrrd, 'seg_%s.nrrd' % seriesuid)
  fn_ct_nrrd = os.path.join(path_nrrd, 'ct_%s.nrrd' % seriesuid)
  fn_param = os.path.join('/content','param_ct.yaml')

  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(fn_param)

  # lung masks
  lungs = fn_seg_nrrd
  lungs, lungsheader = nrrd.read(lungs)

  right_lung = np.zeros(lungs.shape)
  temp =np.where(lungs==1) 
  right_lung[temp]=1

  left_lung = np.zeros(lungs.shape)
  temp =np.where(lungs==2) 
  left_lung[temp]=1

  fn_right_lung = os.path.join(path_nrrd, 'rlung_%s.nrrd' % seriesuid)
  fn_left_lung =  os.path.join(path_nrrd, 'llung_%s.nrrd' % seriesuid)

  nrrd.write(fn_right_lung, right_lung, header = lungsheader)
  nrrd.write(fn_left_lung, left_lung, header = lungsheader)

  # process the lungs
  result = extractor.execute(fn_ct_nrrd, fn_right_lung)
  for key, val in six.iteritems(result):
    print("\t%s: %s" %(key, val))
  fn_out = '/content/radiomics_features_right_lung_%s.csv' % seriesuid
  df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
  df.to_csv(fn_out, index=False)

  result = extractor.execute(fn_ct_nrrd, fn_left_lung)
  for key, val in six.iteritems(result):
    print("\t%s: %s" %(key, val))
  fn_out = '/content/radiomics_features_left_lung_%s.csv' % seriesuid
  df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
  df.to_csv(fn_out, index=False)



1
gsutil cp gs://midrc-analysis-bwh/bpr-results/seg_2.16.840.1.114274.1818.48127916848681376476121209371270597258.nrrd /content/nrrd_data
gsutil cp gs://midrc-analysis-bwh/bpr-results/ct_2.16.840.1.114274.1818.48127916848681376476121209371270597258.nrrd /content/nrrd_data


INFO:radiomics.featureextractor:Loading parameter file /content/param_ct.yaml
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:Applying resampling from spacing [0.72460938 0.72460938 1.        ] and size [512 512 230] to spacing [1. 1. 1.] and size [157, 227, 172]
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextract

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 34385ab4780dba52467af6d5342fb52d604442c9
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.724609375, 0.724609375, 1.0)
	diagnostics_Image-original_Size: (512, 512, 230)
	diagnostics_Image-origin

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


	wavelet-HHH_glrlm_RunPercentage: 0.8422524858709833
	wavelet-HHH_glrlm_RunVariance: 0.2413055376766728
	wavelet-HHH_glrlm_ShortRunEmphasis: 0.881557620911647
	wavelet-HHH_glrlm_ShortRunHighGrayLevelEmphasis: 533.1186749131059
	wavelet-HHH_glrlm_ShortRunLowGrayLevelEmphasis: 0.0015057356668654385
	wavelet-HHH_glszm_GrayLevelNonUniformity: 13955.569713429326
	wavelet-HHH_glszm_GrayLevelNonUniformityNormalized: 0.11555206638428561
	wavelet-HHH_glszm_GrayLevelVariance: 14.70242838614385
	wavelet-HHH_glszm_HighGrayLevelZoneEmphasis: 615.0836859231782
	wavelet-HHH_glszm_LargeAreaEmphasis: 1298499.099773956
	wavelet-HHH_glszm_LargeAreaHighGrayLevelEmphasis: 780437510.6679307
	wavelet-HHH_glszm_LargeAreaLowGrayLevelEmphasis: 2172.8946885102464
	wavelet-HHH_glszm_LowGrayLevelZoneEmphasis: 0.0018116857578425335
	wavelet-HHH_glszm_SizeZoneNonUniformity: 56344.894959966216
	wavelet-HHH_glszm_SizeZoneNonUniformityNormalized: 0.4665355249928893
	wavelet-HHH_glszm_SmallAreaEmphasis: 0.71035994527497

INFO:radiomics.imageoperations:Applying resampling from spacing [0.72460938 0.72460938 1.        ] and size [512 512 230] to spacing [1. 1. 1.] and size [135, 213, 172]
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigma 1
INFO:radiomics.featureextractor:Calculating features for log-sigma-1-0-mm-3D image
INFO:radiomics.featureextractor:Computing

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 34385ab4780dba52467af6d5342fb52d604442c9
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.724609375, 0.724609375, 1.0)
	diagnostics_Image-original_Size: (512, 512, 230)
	diagnostics_Image-origin

In [26]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(fn_param)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

INFO:radiomics.featureextractor:Loading parameter file param_ct.yaml


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
Enabled filters:
	 {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None}


In [20]:
lungs = fn_seg_nrrd
lungs, lungsheader = nrrd.read(lungs)

right_lung = np.zeros(lungs.shape)
temp =np.where(lungs==1) 
right_lung[temp]=1

left_lung = np.zeros(lungs.shape)
temp =np.where(lungs==2) 
left_lung[temp]=1

fn_right_lung = os.path.join(path_nrrd, 'rlung_%s.nrrd' % seriesuid)
fn_left_lung =  os.path.join(path_nrrd, 'llung_%s.nrrd' % seriesuid)

nrrd.write(fn_right_lung, right_lung, header = lungsheader)
nrrd.write(fn_left_lung, left_lung, header = lungsheader)

In [21]:
result = extractor.execute(fn_ct_nrrd, fn_right_lung)
for key, val in six.iteritems(result):
  print("\t%s: %s" %(key, val))

fn_out = '/content/radiomics_features_right_lung.csv'

df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
df.to_csv(fn_out, index=False)

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:Applying resampling from spacing [0.8828125 0.8828125 1.       ] and size [512 512 365] to spacing [1. 1. 1.] and size [204, 233, 275]
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigma 1
INFO:

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 0be8ff22a9782b5ad679573a5dadce1ea0baa18b
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.8828125, 0.8828125, 1.0)
	diagnostics_Image-original_Size: (512, 512, 365)
	diagnostics_Image-original_M

In [22]:
result = extractor.execute(fn_ct_nrrd, fn_left_lung)
for key, val in six.iteritems(result):
  print("\t%s: %s" %(key, val))

fn_out = '/content/radiomics_features_left_lung.csv'

df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
df.to_csv(fn_out, index=False)

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:Applying resampling from spacing [0.8828125 0.8828125 1.       ] and size [512 512 365] to spacing [1. 1. 1.] and size [194, 233, 276]
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigma 1
INFO:

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 0be8ff22a9782b5ad679573a5dadce1ea0baa18b
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.8828125, 0.8828125, 1.0)
	diagnostics_Image-original_Size: (512, 512, 365)
	diagnostics_Image-original_M

In [94]:


fn_out = '/content/radiomics_features.csv'

df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
df.to_csv(fn_out, index=False)


In [ ]:
!pip uninstall highdicom
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

!pip install pydicom
